# 8 · The Appropriate Optics: Why the Logarithmic Scale Is Structural

**Observational record associated with the book**  
*Hidden Structure in the Apparent Chaos of Prime Numbers — Computational experiments through the Euler mirror*  
© Alvaro Costa, 2025

This notebook is part of a canonical sequence of computational records.  
It introduces **no new hypotheses, conjectures, or interpretative models**.

Its sole purpose is to **record** the behaviour of arithmetic structures under an explicit,  
deterministic, and reproducible regime of observation.

The complete conceptual discussion is presented in the book.  
This notebook documents only the corresponding experiment.

**Licence:** Creative Commons BY–NC–ND 4.0  
Reading, execution, and citation are permitted.  
Modification, derivative redistribution, or independent commercial use are not permitted.


---

## 1. The Mystery of the Two Musics

In the previous chapters, we celebrated the emergence of the GOE “music” from our operator $ M $. A closer examination, however, reveals a puzzle: depending  
on how we observe the number line, the very same score sounds different.

When a **linear sampling** is used, the spectrum of $ M $ approaches **Poisson** statistics. Under **logarithmic sampling**, by contrast, the **GOE** emerges  
clearly. Why does the same methodology, applied to the same arithmetic region, produce two such distinct “musics”? Could this be an artefact?

The answer is no. What we are witnessing is a fundamental phenomenon: the way we observe the system changes what the system reveals. The key lies in the natural  
scale of the prime numbers themselves.

---

## 2. The Two Lenses and the Natural Scale of the Primes

The scale that naturally fits the primes is the **logarithmic** one, as Gauss already observed when noting that the density of primes around a number $ x $ is  
approximately $ 1/\ln(x) $. To perceive the global structure of the primes, we need a ruler that expands with them — and that ruler is the logarithmic scale.

Sampling methods therefore act as **two lenses**: one calibrated to this natural scale, and another that ignores it.

---

### The “Panoramic” Lens (Logarithmic Sampling)

This is the natural lens, tuned to the rhythm of the primes. By sampling points at logarithmic intervals, we observe the numerical universe with the appropriate  
ruler — one that grows together with the space itself.

* **What the lens sees:** In this panoramic view, the arithmetic function $ \Delta_\pi(x) $ reveals its harmonic fluctuations and structural noise.
* **The consequence:** The resulting matrix $ M $ has high effective complexity, resembling a real symmetric random matrix, and its spectrum exhibits the  
    characteristic level repulsion of the **GOE (Gaussian Orthogonal Ensemble)**.

---

### The “Microscope” Lens (Linear Sampling)

This lens applies a rigid ruler, without adapting to the logarithmic scale. It is like observing a mountain range through a magnifying glass: everything appears  
flat, because the lens captures only a tiny fragment of the landscape.

* **What the lens sees:** Within such a narrow region, the global trend disappears and $ \Delta_\pi(x) $ appears stable and smooth. Local fluctuations become almost  
    independent — a regime of **decorrelation**.
* **The consequence:** The matrix ( M ) constructed through this lens has low variability, and its spectrum behaves like a system of independent events — a **Poisson**  
    statistic.

---

### Comparative Table

| Feature                  | Linear Sampling (Microscope)            | Logarithmic Sampling (Panoramic)       |
| ------------------------ | --------------------------------------- | -------------------------------------- |
| **Alignment**            | Ignores the natural scale of the primes | Aligned with the natural scale (Gauss) |
| **View of the function** | Locally “flat” region                   | Global fluctuations and harmonic noise |
| **Matrix complexity**    | Low, ordered, predictable               | High, complex, pseudo-random           |
| **Spectral outcome**     | **Poisson**                             | **GOE (Gaussian Orthogonal Ensemble)** |

---

## 3. The Scale Laboratory: Connection with the Code

The code cell below demonstrates this effect directly.  
One technical detail is essential to reveal the Poisson structure: the function `local_normalize_spacings`.

Because the linear lens observes only a “nearly flat” region, the density of eigenvalues may vary slightly.  
Local normalisation acts as a **fine-focus adjustment** — it rescales each spacing according to its immediate neighbourhood, restoring a faithful view of the Poisson  
regime.

This procedure does not create the pattern; it reveals it.  
When `local_normalize_spacings` is applied, the Poisson distribution appears sharply — demonstrating that it is a **genuine property of the system at this scale**,  
not a numerical artefact.


In [1]:
# Requirements: pandas, matplotlib, numpy, ipywidgets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time

# --- Data Generation and Matrix Functions ---
def generate_pi_data(n: int) -> np.ndarray:
    """Generates an array containing all primes up to n using an optimised sieve."""
    if n < 2: return np.array([], dtype=np.int64)
    size = (n - 1) // 2; sieve = np.ones(size, dtype=bool)
    limit = int(np.sqrt(n)) // 2
    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3; start = (p*p - 3) // 2
            sieve[start::p] = False
    indices = np.where(sieve)[0]; odd_primes = 2 * indices + 3
    return np.concatenate((np.array([2], dtype=np.int64), odd_primes))

def get_delta_pi_for_points(x_points, primes):
    """Computes Δπ(x) for an array of x-points using a precomputed list of primes."""
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2
    
def generate_cos_matrix(fx_values, x_values):
    """Generates the matrix M from the vectors F(x) and x."""
    fx = fx_values.astype(np.float64); x = x_values.astype(np.float64)
    x[x <= 0] = 1e-12; logx = np.log(x)
    C = np.cos(np.outer(fx, logx)); M = C + C.T
    std_dev = M.std()
    if std_dev > 0:
        M -= M.mean()
        M /= std_dev
    return 0.5 * (M + M.T)

# Fixed bulk: central 90% (alpha = 0.10)
# Fixed local window for unfolding (not optimised): w = 21
def local_normalize_spacings(lam, alpha=0.10, w=21):
    """
    Normalises spacings by their local mean (unfolding).
    This is the key step to correctly visualise Poisson statistics.
    """
    N = lam.size
    # Extract the spectral bulk to avoid edge effects
    k0, k1 = int(alpha * N), int((1 - alpha) * N)
    lam_bulk = np.sort(lam)[k0:k1]
    
    s = np.diff(lam_bulk)
    s = s[s > 0]
    
    if len(s) < w: 
        return s / s.mean() if s.mean() > 0 else s

    # Use a moving average to estimate the local density of states
    w = int(w)
    if w % 2 == 0: w += 1  # Window length must be odd
    pad = w // 2
    s_padded = np.pad(s, (pad, pad), mode='reflect')
    local_mean = np.convolve(s_padded, np.ones(w)/w, mode='valid')
    
    # Avoid division by zero
    local_mean[local_mean == 0] = 1.0
    
    return s / local_mean

# --- Main Interactive Function ---
def scale_comparison_lab(N=2048, log_X0=8, span=2.4):
    
    X0 = int(10**log_X0)
    
    # --- Data Preparation ---
    max_x_log = int(np.ceil(X0 * np.exp(span/2)))
    max_x_linear = X0 + N
    max_x_needed = max(max_x_log, max_x_linear)
    pi_x_full = generate_pi_data(max_x_needed)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True) 
    
    # --- Left Plot: Linear Sampling (Poisson) ---
    print("\n--- Processing Linear Scale ---")
    x_linear = np.arange(X0, X0 + N)
    fx_linear = get_delta_pi_for_points(x_linear, pi_x_full)
    
    M_linear = generate_cos_matrix(fx_linear, x_linear)
    lam_linear, _ = np.linalg.eigh(M_linear)
    # USE LOCAL NORMALISATION TO REVEAL POISSON
    s_unfolded_linear = local_normalize_spacings(lam_linear)

    # --- Right Plot: Logarithmic Sampling (GOE) ---
    print("\n--- Processing Logarithmic Scale ---")
    x_log = np.exp(np.linspace(np.log(X0) - span/2, np.log(X0) + span/2, N))
    fx_log = get_delta_pi_for_points(x_log, pi_x_full)

    M_log = generate_cos_matrix(fx_log, x_log)
    lam_log, _ = np.linalg.eigh(M_log)
    # For GOE, global mean normalisation is sufficient
    s_log = np.diff(np.sort(lam_log)); s_log = s_log[s_log > 0]
    s_unfolded_log = s_log / s_log.mean()

    # --- Comparative Plots ---
    s_grid = np.linspace(0, 4, 200)
    pdf_goe = (np.pi * s_grid / 2) * np.exp(-np.pi * s_grid**2 / 4)
    pdf_poisson = np.exp(-s_grid)
    
    # Left plot
    ax = axes[0]
    ax.hist(s_unfolded_linear, bins='auto', density=True, alpha=0.75, label='Data (Linear)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='GOE Theory')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Poisson Theory')
    ax.set_title('a) Linear Scale → Uncorrelated Regime', fontsize=14)
    ax.set_xlabel('s (Locally Normalised Spacing)'); ax.set_ylabel('Density')
    ax.set_xlim(0, 4); ax.legend(loc='upper right')
    
    # Right plot
    ax = axes[1]
    ax.hist(s_unfolded_log, bins='auto', density=True, alpha=0.75, label='Data (Logarithmic)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='GOE Theory')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Poisson Theory')
    ax.set_title('b) Logarithmic Scale → Correlated Regime', fontsize=14)
    ax.set_xlabel('s (Globally Normalised Spacing)'); ax.legend(loc='upper right')
    ax.set_xlim(0, 4)
    
    fig.suptitle(
        f"Visual Comparison of the Effect of Scale at X₀ = {X0:g}",
        fontsize=18, weight='bold'
    )
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# --- Interactive Widget ---
interact(
    scale_comparison_lab, 
    N=widgets.Dropdown(options=[512, 1024, 2048], value=2048, description='N:'),
    log_X0=widgets.IntSlider(
        min=5, max=8, step=1, value=8,
        description='X₀=10^', continuous_update=False
    ),
    span=widgets.FloatSlider(
        min=1.0, max=4.0, step=0.1,
        value=2.4, description='Span (Log):'
    )
);


interactive(children=(Dropdown(description='N:', index=2, options=(512, 1024, 2048), value=2048), IntSlider(va…

> With experimental proof in hand that the two lenses produce different kinds of music, we are ready to go deeper. In the next chapter, we will examine the *optics*  
> of our logarithmic lens, delving into the mathematics that explains why it does not merely work, but why it is the **only** lens that could work.
